In [0]:
# Avirut Mehta
# 200319@0045

In [0]:
import pandas as pd
import numpy as np

In [0]:
# source: https://www.washingtonpost.com/wp-stat/dea-pain-pill-database/summary/arcos-wv-statewide-itemized.csv.gz
data = pd.read_csv('/content/drive/My Drive/Global Health Project/Analysis/arcos-wv-statewide-itemized.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,5,18,21,24,26,37,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# raw data has a county "25701" that should be Cabell County based on actual address of purchases
# but these 7 transactions have no date so let's just drop them entirely
data = data.drop(data.index[data['BUYER_COUNTY']=='25701'].tolist())

In [0]:
rawcounties = []
years = []
for county in data['BUYER_COUNTY']:
  if not (county in rawcounties):
    rawcounties.append(county)
for date in data['TRANSACTION_DATE']:
  try:
    thisYear = int(str(date)[-6:-2])
  except ValueError:
    print(date)
  if not (thisYear in years):
    years.append(thisYear)

In [0]:
counties = []
numYears = len(years)
years = years*len(rawcounties);
counties = [[county.lower().capitalize()]*numYears for county in rawcounties]
counties = [item for sublist in counties for item in sublist]

In [0]:
dIndex = pd.MultiIndex.from_arrays([counties, years], names=['County', 'Year'])
cols = ['NumTrans', 'TotMME', 'TotDU']
d = pd.DataFrame(np.zeros((len(counties), len(cols))), index=dIndex, columns=cols)

In [0]:
# NumTrans will be the number of total recorded transactions per county
# TotMME is the total milligram morphine equivalent 
# TotDU is total quantity pills, etc.
# est running time = ~1.5 hours

numRows = len(data)
for index, row in data.iterrows():
  if index%100==0:
    print('\r', 'Current index:', str(index), 'out of', str(numRows), end='')
  thisCounty = row['BUYER_COUNTY'].lower().capitalize()
  thisYear = int(str(row['TRANSACTION_DATE'])[-6:-2])
  d.loc[(thisCounty, thisYear), 'NumTrans'] += 1
  d.loc[(thisCounty, thisYear), 'TotMME'] += row['MME']
  d.loc[(thisCounty, thisYear), 'TotDU'] += row['DOSAGE_UNIT']

 Current index: 2419200 out of 2419237

In [0]:
d = d.sort_values(['County', 'Year'])

In [0]:
d.to_csv('/content/drive/My Drive/Global Health Project/Analysis/arcos_sanitized_am_200319.csv')